In [1]:
import ROOT
ROOT.TMVA.Tools.Instance()

Welcome to ROOTaaS 6.06/06


<ROOT.TMVA::Tools object at 0x4f62c40>

In [2]:
outfileName = "KNN_Test_via_TMVAReg.root"
outputFile = ROOT.TFile(outfileName,"RECREATE")
factory = ROOT.TMVA.Factory("TMVARegression", outputFile,
                            "!V:!Silent:Color:DrawProgressBar" )

--- Factory                  : You are running ROOT Version: 6.06/06, Jul 6, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


In [4]:
# Define the input variables that shall be used for the MVA training
# note that you may also use variable expressions, such as: "3*var1/var2*abs(var3)"
# [all types of expressions that can also be parsed by TTree::Draw( "expression" )]
factory.AddVariable( "var1", "Variable 1", "units", 'F' )
factory.AddVariable( "var2", "Variable 2", "units", 'F' )

# You can add so-called "Spectator variables", which are not used in the MVA training, 
# but will appear in the final "TestTree" produced by TMVA. This TestTree will contain the 
# input variables, the response values of all trained MVAs, and the spectator variables
factory.AddSpectator( "spec1:=var1*2",  "Spectator 1", "units", -100.0, 100.0  )
factory.AddSpectator( "spec2:=var1*3",  "Spectator 2", "units", -100.0, 100.0  )

# Add the variable carrying the regression target
factory.AddTarget( "fvalue" )

# It is also possible to declare additional targets for multi-dimensional regression, ie:
# -- factory->AddTarget( "fvalue2" );
# BUT: this is currently ONLY implemented for MLP

In [5]:
fname = "./tmva_reg_example.root";
inputFile = ROOT.TFile.Open( fname );

In [9]:
# --- Register the regression tree

regTree = inputFile.Get("TreeR");

# global event weights per tree (see below for setting event-wise weights)
regWeight  = 1.0;   

# You can add an arbitrary number of regression trees
factory.AddRegressionTree( regTree, regWeight );

# This would set individual event weights (the variables defined in the 
# expression need to exist in the original TTree)
factory.SetWeightExpression( "var1", "Regression" );

# Apply additional cuts on the signal and background samples (can be different)
mycut = ROOT.TCut('') # for example: TCut mycut = "abs(var1)<0.5 && abs(var2-0.5)<1";

# tell the factory to use all remaining events in the trees after training for testing:
factory.PrepareTrainingAndTestTree( mycut, 
                                    "nTrain_Regression=1000:nTest_Regression=0:SplitMode=Random:NormMode=NumEvents:!V" )
# factory.PrepareTrainingAndTestTree( mycut, 
#                                   "nTrain_Regression=0:nTest_Regression=0:SplitMode=Random:NormMode=NumEvents:!V" );

# If no numbers of events are given, half of the events in the tree are used 
# for training, and the other half for testing:
#    factory.PrepareTrainingAndTestTree( mycut, "SplitMode=random:!V" )

--- Factory                  : Add Tree TreeR of type Regression with 10000 events
--- DataSetInfo              : Class index : 0  name : Regression


In [11]:
# Book the actual method
factory.BookMethod( ROOT.TMVA.Types.kMLP, "MLP",
                   "!H:!V:VarTransform=Norm:NeuronType=tanh:NCycles=20000:HiddenLayers=N+20:TestRate=6:TrainingMethod=BFGS:Sampling=0.3:SamplingEpoch=0.8:ConvergenceImprove=1e-6:ConvergenceTests=15:!UseRegulator" );

--- Factory                  : Booking method: MLP
--- MLP                      : Create Transformation "Norm" with events from all classes.
--- Norm                     : Transformation, Variable selection : 
--- Norm                     : Input : variable 'var1' (index=0).   <---> Output : variable 'var1' (index=0).
--- Norm                     : Input : variable 'var2' (index=1).   <---> Output : variable 'var2' (index=1).
--- Norm                     : Input : variable 'var1' (index=2).   <---> Output : variable 'var1' (index=2).
--- Norm                     : Input : variable 'var2' (index=3).   <---> Output : variable 'var2' (index=3).
--- Norm                     : Input : target 'fvalue' (index=0).   <---> Output : target 'fvalue' (index=0).
--- MLP                      : Building Network
--- MLP                      : Initializing weights


In [ ]:
# Train MVAs using the set of training events
factory.TrainAllMethods();

# ---- Evaluate all MVAs using the set of test events
factory.TestAllMethods();

# ----- Evaluate and compare performance of all configured MVAs
factory.EvaluateAllMethods();  

In [ ]:
outputFile.Close()
print("==> Wrote root file: " + outputFile.GetName())
print("==> TMVAClassification is done!")
del factory
ROOT.TMVA.TMVAGui( outfileName )

In [1]:
raw_input("Press Enter to continue...")

Press Enter to continue...


''